<a href="https://colab.research.google.com/github/atakhadivi/Algorithmic-Trading/blob/master/aroonchaikinoscilator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install everything we need to import
!pip3 install -q python-binance
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
!pip3 install -q tinydb

In [ ]:
# import every thing need to bot work
import datetime
from datetime import date
from binance.client import Client
import pandas as pd
import talib as ta
import numpy as np
from tinydb import TinyDB, Query
import time
import requests
from binance.enums import *

In [ ]:
# api key and api secret
api_key = 'HrH5Zgni8Qhvu9yk5gniThkUWShZAPymSfgELlukZ29dn5PD5qg0wZkc68uJAtAr'
api_secret = '6xuDDQEDAmg8EmWAckER5vYG7ewM5eDR2luzAgClLVfKT6U47qXFMMfvW54ZVKC0'

In [ ]:
# all symbols for tradiong
symbol = [
  "BTCUSDT",
  "ETHUSDT",
  "LTCUSDT",
  "BNBUSDT",
  "NEOUSDT",
  "QTUMUSDT",
  "ADAUSDT",
  "XRPUSDT",
  "EOSUSDT",
  "IOTAUSDT",
  "XLMUSDT",
  "ONTUSDT",
  "TRXUSDT",
  "ETCUSDT",
  "ICXUSDT",
  "NULSUSDT",
  "VETUSDT",
  "LINKUSDT",
  "WAVESUSDT",
  "BTTUSDT",
  "ONGUSDT",
  "HOTUSDT",
  "ZILUSDT",
  "ZRXUSDT",
  "FETUSDT",
  "BATUSDT",
  "XMRUSDT",
  "ZECUSDT",
  "IOSTUSDT",
  "CELRUSDT",
  "DASHUSDT",
  "NANOUSDT",
  "OMGUSDT",
  "THETAUSDT",
  "ENJUSDT",
  "MITHUSDT",
  "MATICUSDT",
  "ATOMUSDT",
  "TFUELUSDT",
  "ONEUSDT",
  "FTMUSDT",
  "ALGOUSDT",
  "GTOUSDT",
  "ERDUSDT",
  "DOGEUSDT",
  "DUSKUSDT",
  "ANKRUSDT",
  "WINUSDT",
  "COSUSDT",
  "NPXSUSDT",
  "COCOSUSDT",
  "MTLUSDT",
  "TOMOUSDT",
  "PERLUSDT",
  "DENTUSDT",
  "MFTUSDT",
  "KEYUSDT",
  # "STORMUSDT",
  "DOCKUSDT",
  "WANUSDT",
  "FUNUSDT",
  "CVCUSDT",
  "CHZUSDT",
  "BANDUSDT",
  "BEAMUSDT",
  "XTZUSDT",
  "RENUSDT",
  "RVNUSDT",
  "HCUSDT",
  "HBARUSDT",
  "NKNUSDT",
  "STXUSDT",
  "KAVAUSDT",
  "ARPAUSDT",
  "IOTXUSDT",
  "RLCUSDT",
  "MCOUSDT",
  "CTXCUSDT",
  "BCHUSDT",
  "TROYUSDT",
  "VITEUSDT",
  "FTTUSDT",
  "OGNUSDT",
  "DREPUSDT",
  "TCTUSDT",
  "WRXUSDT",
  "BTSUSDT",
  "LSKUSDT",
  "BNTUSDT",
  "LTOUSDT",
  "STRATUSDT",
  "AIONUSDT",
  "MBLUSDT",
  "COTIUSDT",
  "STPTUSDT",
  "WTCUSDT",
  "DATAUSDT",
  "XZCUSDT",
]

In [ ]:
rules = {
  "BTCUSDT":"%.6f",
  "ETHUSDT":"%.5f",
  "LTCUSDT":"%.5f",
  "BNBUSDT":"%.2f",
  "NEOUSDT":"%.3f",
  "QTUMUSDT":"%.3f",
  "ADAUSDT":"%.1f",
  "XRPUSDT":"%.1f",
  "EOSUSDT":"%.2f",
  "IOTAUSDT":"%.2f",
  "XLMUSDT":"%.1f",
  "ONTUSDT":"%.2f",
  "TRXUSDT":"%.1f",
  "ETCUSDT":"%.2f",
  "ICXUSDT":"%.2f",
  "NULSUSDT":"%.2f",
  "VETUSDT":"%.0f",
  "LINKUSDT":"%.2f",
  "WAVESUSDT":"%.2f",
  "BTTUSDT":"%.0f",
  "ONGUSDT":"%.2f",
  "HOTUSDT":"%.0f",
  "ZILUSDT":"%.1f",
  "ZRXUSDT":"%.2f",
  "FETUSDT":"%.1f",
  "BATUSDT":"%.2f",
  "XMRUSDT":"%.5f",
  "ZECUSDT":"%.5f",
  "IOSTUSDT":"%.0f",
  "CELRUSDT":"%.1f",
  "DASHUSDT":"%.5f",
  "NANOUSDT":"%.2f",
  "OMGUSDT":"%.2f",
  "THETAUSDT":"%.1f",
  "ENJUSDT":"%.1f",
  "MITHUSDT":"%.1f",
  "MATICUSDT":"%.1f",
  "ATOMUSDT":"%.3f",
  "TFUELUSDT":"%.0f",
  "ONEUSDT":"%.1f",
  "FTMUSDT":"%.1f",
  "ALGOUSDT":"%.2f",
  "GTOUSDT":"%.1f",
  "ERDUSDT":"%.0f",
  "DOGEUSDT":"%.0f",
  "DUSKUSDT":"%.2f",
  "ANKRUSDT":"%.0f",
  "WINUSDT":"%.0f",
  "COSUSDT":"%.1f",
  "NPXSUSDT":"%.0f",
  "COCOSUSDT":"%.0f",
  "MTLUSDT":"%.2f",
  "TOMOUSDT":"%.2f",
  "PERLUSDT":"%.1f",
  "DENTUSDT":"%.0f",
  "MFTUSDT":"%.0f",
  "KEYUSDT":"%.0f",
  "STORMUSDT":"%.0f",
  "DOCKUSDT":"%.0f",
  "WANUSDT":"%.2f",
  "FUNUSDT":"%.0f",
  "CVCUSDT":"%.1f",
  "CHZUSDT":"%.0f",
  "BANDUSDT":"%.2f",
  "BEAMUSDT":"%.2f",
  "XTZUSDT":"%.2f",
  "RENUSDT":"%.1f",
  "RVNUSDT":"%.1f",
  "HCUSDT":"%.2f",
  "HBARUSDT":"%.1f",
  "NKNUSDT":"%.1f",
  "STXUSDT":"%.2f",
  "KAVAUSDT":"%.2f",
  "ARPAUSDT":"%.1f",
  "IOTXUSDT":"%.0f",
  "RLCUSDT":"%.2f",
  "MCOUSDT":"%.3f",
  "CTXCUSDT":"%.2f",
  "BCHUSDT":"%.5f",
  "TROYUSDT":"%.0f",
  "VITEUSDT":"%.1f",
  "FTTUSDT":"%.3f",
  "OGNUSDT":"%.2f",
  "DREPUSDT":"%.0f",
  "TCTUSDT":"%.0f",
  "WRXUSDT":"%.1f",
  "BTSUSDT":"%.1f",
  "LSKUSDT":"%.2f",
  "BNTUSDT":"%.2f",
  "LTOUSDT":"%.1f",
  "STRATUSDT":"%.2f",
  "AIONUSDT":"%.2f",
  "MBLUSDT":"%.0f",
  "COTIUSDT":"%.1f",
  "STPTUSDT":"%.1f",
  "WTCUSDT":"%.2f",
  "DATAUSDT":"%.1f",
  "XZCUSDT":"%.3f",
  "SOLUSDT":"%.2f"
}


In [ ]:
price = {
  "BTCUSDT":"%.2f",
  "ETHUSDT":"%.2f",
  "LTCUSDT":"%.2f",
  "BNBUSDT":"%.4f",
  "NEOUSDT":"%.3f",
  "QTUMUSDT":"%.3f",
  "ADAUSDT":"%.1f",
  "XRPUSDT":"%.1f",
  "EOSUSDT":"%.4f",
  "IOTAUSDT":"%.4f",
  "XLMUSDT":"%.1f",
  "ONTUSDT":"%.4f",
  "TRXUSDT":"%.5f",
  "ETCUSDT":"%.4f",
  "ICXUSDT":"%.4f",
  "NULSUSDT":"%.4f",
  "VETUSDT":"%.6f",
  "LINKUSDT":"%.4f",
  "WAVESUSDT":"%.4f",
  "BTTUSDT":"%.6f",
  "ONGUSDT":"%.4f",
  "HOTUSDT":"%.6f",
  "ZILUSDT":"%.5f",
  "ZRXUSDT":"%.4f",
  "FETUSDT":"%.5f",
  "BATUSDT":"%.4f",
  "XMRUSDT":"%.2f",
  "ZECUSDT":"%.2f",
  "IOSTUSDT":"%.6f",
  "CELRUSDT":"%.5f",
  "DASHUSDT":"%.2f",
  "NANOUSDT":"%.4f",
  "OMGUSDT":"%.4f",
  "THETAUSDT":"%.5f",
  "ENJUSDT":"%.5f",
  "MITHUSDT":"%.5f",
  "MATICUSDT":"%.5f",
  "ATOMUSDT":"%.3f",
  "TFUELUSDT":"%.6f",
  "ONEUSDT":"%.5f",
  "FTMUSDT":"%.5f",
  "ALGOUSDT":"%.4f",
  "GTOUSDT":"%.5f",
  "ERDUSDT":"%.6f",
  "DOGEUSDT":"%.6f",
  "DUSKUSDT":"%.4f",
  "ANKRUSDT":"%.6f",
  "WINUSDT":"%.6f",
  "COSUSDT":"%.5f",
  "NPXSUSDT":"%.6f",
  "COCOSUSDT":"%.6f",
  "MTLUSDT":"%.4f",
  "TOMOUSDT":"%.4f",
  "PERLUSDT":"%.5f",
  "DENTUSDT":"%.6f",
  "MFTUSDT":"%.6f",
  "KEYUSDT":"%.6f",
  "STORMUSDT":"%.6f",
  "DOCKUSDT":"%.6f",
  "WANUSDT":"%.4f",
  "FUNUSDT":"%.6f",
  "CVCUSDT":"%.5f",
  "CHZUSDT":"%.6f",
  "BANDUSDT":"%.4f",
  "BEAMUSDT":"%.4f",
  "XTZUSDT":"%.4f",
  "RENUSDT":"%.5f",
  "RVNUSDT":"%.5f",
  "HCUSDT":"%.4f",
  "HBARUSDT":"%.5f",
  "NKNUSDT":"%.5f",
  "STXUSDT":"%.4f",
  "KAVAUSDT":"%.4f",
  "ARPAUSDT":"%.5f",
  "IOTXUSDT":"%.6f",
  "RLCUSDT":"%.4f",
  "MCOUSDT":"%.3f",
  "CTXCUSDT":"%.4f",
  "BCHUSDT":"%.2f",
  "TROYUSDT":"%.6f",
  "VITEUSDT":"%.5f",
  "FTTUSDT":"%.3f",
  "OGNUSDT":"%.4f",
  "DREPUSDT":"%.6f",
  "TCTUSDT":"%.6f",
  "WRXUSDT":"%.5f",
  "BTSUSDT":"%.5f",
  "LSKUSDT":"%.4f",
  "BNTUSDT":"%.4f",
  "LTOUSDT":"%.5f",
  "STRATUSDT":"%.4f",
  "AIONUSDT":"%.4f",
  "MBLUSDT":"%.6f",
  "COTIUSDT":"%.5f",
  "STPTUSDT":"%.5f",
  "WTCUSDT":"%.4f",
  "DATAUSDT":"%.5f",
  "XZCUSDT":"%.3f",
  "SOLUSDT":"%.4f"
  }



In [ ]:
# database creation
db = TinyDB('db.json')
user = Query()
for i in symbol:
  db.insert({'symbol':i,'Volume':10})

In [ ]:
try:
  while True:
    for i in symbol:
      milli_sec = time.time() * 1000
      sec = int(milli_sec) - 86400000
      client = Client(api_key, api_secret)
      df = client.get_historical_klines(i, Client.KLINE_INTERVAL_1DAY, "30 day ago UTC")
      df = pd.DataFrame(df,columns= ['datetime', 'open', 'high', 'low', 'close', 'volume', 'Close_time','qoute assest volume','number of trades','taker buy base assest volume','taker buy qoute assest volume','ignore'])
      df['datetime'] = pd.to_datetime(df['datetime'],unit = 'ms')
      df.set_index('datetime',inplace=True)
      df = pd.DataFrame(data=df, dtype=np.float64)
      close = df['close'].values
      high = df['high'].values
      low = df['low'].values
      volume = df['volume'].values
      rl = db.get(user.symbol == i)['Volume']
      # print(i,rl)
      real = ta.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
      real = [x for x in real if str(x) != 'nan']
      real_length = len(real)
      real_now = real[real_length - 1]
      real_old = real[real_length - 2]
      real_older = real[real_length - 3]
      real_oldest = real[real_length - 4]
      aroondown, aroonup = ta.AROON(high, low, timeperiod=25)
      aroonup_length = len(aroonup)
      real_aroon = aroonup[aroonup_length - 1]
      # print(i,real_aroon)
      # print(real_old)
      sell_price = df['close'].tail(1).values + ((df['close'].tail(1).values / 100) * 2.00)
      stop_loss = df['close'].tail(1).values - ((df['close'].tail(1).values / 100) * 2.00)
      # print(real_now)
      if float(real_now) > 0 and float(real_aroon) > 70:# and float(db.get(user.symbol == i)['Volume']) < 0:
        # if float(db.get(user.symbol == i)['Volume']) < 0:
            if real_old > 0 and real_older < 0 and real_oldest < 0:
              # print(i,"BUY",real_old,real_older)
              client = Client(api_key, api_secret)
              #get btc balance
              balance = client.get_asset_balance(asset='USDT')
              balance = float(balance['free'])
              assest_price = df['close'].tail(1).values
              # return(balance)
              quantity = 50 / assest_price
              # quantity = float(quantity)
              sell_price = float(sell_price)
              stop_loss = float(stop_loss)
              orders = client.get_open_orders(symbol=i)
              # print(orders)
              all_orders = client.get_all_orders(symbol=i,startTime=sec)
              all_orders_length = len(all_orders)
              if len(all_orders) > 0:
                all_orders_status = all_orders[all_orders_length - 1]['status']
                if not orders and "FILLED" in all_orders_status:
                  if (float(balance) > 10):
                      order = client.order_market_buy(symbol=i, quantity=rules[i] % float(quantity))
                      print(order)
                      asset_balance = order["executedQty"]
                      print(asset_balance)
                      # order = client.order_limit_sell(
                      #     symbol=i,
                      #     quantity=rules[i] % float(asset_balance),
                      #     price=price[i] % sell_price)
                      # print(order)
                      order = client.create_oco_order(
                        symbol=i,
                        side="SELL",
                        stopLimitTimeInForce="GTC",
                        quantity=rules[i] % float(asset_balance),
                        stopPrice=price[i] % stop_loss,
                        stopLimitPrice=price[i] % stop_loss,
                        price=price[i] % sell_price)
              elif not orders:# and "FILLED" in all_orders_status:
                if (float(balance) > 10):
                    order = client.order_market_buy(symbol=i, quantity=rules[i] % float(quantity))
                    print(order)
                    asset_balance = order["executedQty"]
                    print(asset_balance)
                    # order = client.order_limit_sell(
                    #     symbol=i,
                    #     quantity=rules[i] % float(asset_balance),
                    #     price=price[i] % sell_price)
                    # print(order)
                    order = client.create_oco_order(
                      symbol=i,
                      side="SELL",
                      stopLimitTimeInForce="GTC",
                      quantity=rules[i] % float(asset_balance),
                      stopPrice=price[i] % stop_loss,
                      stopLimitPrice=price[i] % stop_loss,
                      price=price[i] % sell_price)
      # db.update({'Volume':real_now}, user.symbol == i)
except ValueError as ve:
  print(ve)